In [1]:
import numpy as np
import pickle
from helpers import * 
import properscoring as ps

In [2]:
season = 'JAS'
data_dir = "../../precip_data"
save_full = True

In [3]:
data_save = data_dir + '/results/'
data_save2 = data_dir + '/results/full_models/'
folds = np.arange(9)
lsm = np.loadtxt(data_dir  + "/lsm.txt")

crps_fold = np.zeros(len(folds))
bs_fold = np.zeros(len(folds))
for fold in folds:
    print(fold)
    crps_cv = np.zeros((19, 61))
    bs_cv = np.zeros((19, 61))
    ytrain, time_train = load_obs_time(data_dir, fold, mode = "train")
    if fold == 8:
        yval, time_test = load_obs_time(data_dir, fold, mode = "test")
    else:
        yval, time_test = load_obs_time(data_dir, fold, mode = "val")

    # array with time x lat x lon
    year_dim = yval.shape[0]
    ix0, ix1 = sel_season_indx(season, year_dim)
    time_test_season = time_test[ix0:ix1]
    unique_tts = np.unique(time_test_season)
    if save_full:
        model_bs = xr.DataArray(
               np.random.rand(19, 61),
               coords=[np.arange(19) ,np.arange(-25, 35.5) ],
               dims=["lat", "lon"],
               name='var'
            )
        model_crps = xr.DataArray(
               np.random.rand(19, 61),
               coords=[np.arange(19) ,np.arange(-25, 35.5) ],
               dims=["lat", "lon"],
               name='var'
            )
            
    for i in range(19):
        for j in range(61):
            if lsm[i, j] != 0:
                ytrain_grid = ytrain[:, i, j] 
                yval_grid = yval[ix0:ix1, i, j]
                crps_ens = list()
                bs_ens = list()
                for val in unique_tts:
                    ix_month = np.where(val == time_train)[0]
                    month_ens = ytrain_grid[ix_month]
                    month_prob = np.mean(month_ens > 0.2)
                    yval_grid_month = yval_grid[val == time_test_season]
                    ens_dim = len(yval_grid_month)    
                    ens_mat = np.tile(month_ens, (ens_dim, 1))
                    crps_ens.append(ps.crps_ensemble(yval_grid_month, ens_mat))
                    yval_grid_month_bin = yval_grid_month > 0.2
                    ens_prob = np.repeat(month_prob, ens_dim)
                    bs_ens.append(np.square(yval_grid_month_bin - ens_prob))

                crps_cv[i,j] = np.mean(np.concatenate(crps_ens))
                bs_cv[i, j] = np.mean(np.concatenate(bs_ens))
            else:
                crps_cv[i,j] = np.nan
                bs_cv[i, j] = np.nan
            
    crps_fold[fold] = np.nanmean(crps_cv)
    bs_fold[fold] = np.nanmean(bs_cv)

    if save_full:
        model_bs[:, :] = bs_cv
        model_bs.to_netcdf(data_save2 +  'mpc_bs_'+ season + '_' + str(fold) +'.nc')
        model_crps[:, :] = crps_cv
        model_crps.to_netcdf(data_save2  + 'mpc_crps_'+ season + '_' + str(fold) +'.nc')
        
np.savetxt(data_save  + 'mpc_crps_'+ season +  '.txt', crps_fold)
np.savetxt(data_save  + 'mpc_bs_'+ season +  '.txt', bs_fold)

0
1
2
3
4
5
6
7
8


In [3]:
bs_fold

array([0.1671104 , 0.15877314, 0.16606004, 0.16796413, 0.1740587 ,
       0.15887017, 0.16561248, 0.16065015, 0.1536647 ])

In [6]:
crps_fold

array([3.35477721, 3.54058059, 3.09141036, 3.30758144, 3.14746003,
       3.33223891, 3.24978596, 3.26668665, 3.31891571])